# Frequent Items

## Initialization
- Load cleaned data and user data with clusters

In [100]:
import pandas as pd
import numpy as np
import os
from sklearn.neighbors import NearestNeighbors
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from mlxtend.frequent_patterns import apriori, association_rules
from sklearn.preprocessing import MultiLabelBinarizer
from scipy.sparse import csr_matrix
from mlxtend.frequent_patterns import fpgrowth
from collections import Counter
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from mlxtend.preprocessing import TransactionEncoder
from collections import defaultdict
from itertools import combinations
import networkx as nx
import ast
from sklearn.metrics.pairwise import cosine_similarity
import warnings
# Ignore all warnings
warnings.filterwarnings("ignore")

In [101]:
file_path_songs = "Data/df_combined.csv"
df_songs = pd.read_csv(file_path_songs)
file_path_clusters = "Data/df_users.csv"
df_users = pd.read_csv(file_path_clusters)

## Compute market basket data frames
- A market basket data frame is computed for each cluster
- Only users with more than 50 total listenings are included
- Each basket includes at most the users 100 most played songs
- Songs which does not occur in more than 10 different baskets are excluded

We generate 11 separate DataFrames stored as `df_basket_1`, `df_basket_2`, ..., `df_basket_11`.

In [102]:
# Calculate the total play count for each user
user_total_playcount = df_users.groupby('user_id')['playcount'].sum().reset_index()
user_total_playcount = user_total_playcount[user_total_playcount['playcount'] >= 50]

# Filter the original user data to include only users with at least 50 total plays
df_users_filtered = df_users[df_users['user_id'].isin(user_total_playcount['user_id'])]

# Process each cluster to generate individual DataFrames
cluster_dataframes = {}  # Dictionary to hold the dataframes for each cluster

for cluster_id in sorted(df_users_filtered['most_played_cluster'].unique()[:11]):  # Ensure max 11 clusters
    # Filter the data for the current cluster
    cluster_data = df_users_filtered[df_users_filtered['most_played_cluster'] == cluster_id]

    # Sort songs for each user by play count and select the 100 most played songs
    def get_top_songs(user_data):
        return user_data.nlargest(100, 'playcount')  # Adjust to desired number of top songs

    # Apply the top song selection for each user
    df_top_songs_cluster = cluster_data.groupby('user_id').apply(get_top_songs).reset_index(drop=True)

    # Create a basket for each user with their top songs
    df_basket_cluster = df_top_songs_cluster.groupby('user_id')['track_id'].apply(list).reset_index()
    df_basket_cluster.rename(columns={'track_id': 'basket'}, inplace=True)

    # Assign the DataFrame to a variable dynamically
    cluster_dataframes[f"df_basket_{cluster_id}"] = df_basket_cluster

    # Optionally, save to a named variable
    globals()[f"df_basket_{cluster_id}"] = df_basket_cluster

In [103]:
# Function to filter songs occurring in more than 10 baskets
def filter_songs(df_basket, min_basket_count=10):
    """
    Filters out songs from the baskets that occur in fewer than `min_basket_count` baskets.
    
    Parameters:
    - df_basket (pd.DataFrame): DataFrame with columns `user_id` and `basket`.
    - min_basket_count (int): Minimum number of baskets a song must appear in to be retained.

    Returns:
    - pd.DataFrame: Filtered DataFrame with updated baskets.
    """
    # Flatten all baskets into a single list to count song occurrences
    all_songs = [song for basket in df_basket['basket'] for song in basket]
    song_counts = Counter(all_songs)

    # Identify songs that occur in more than `min_basket_count` baskets
    frequent_songs = {song for song, count in song_counts.items() if count > min_basket_count}

    # Filter each user's basket to retain only frequent songs
    df_basket['basket'] = df_basket['basket'].apply(lambda basket: [song for song in basket if song in frequent_songs])

    # Remove rows with empty baskets
    df_basket = df_basket[df_basket['basket'].map(len) > 0].reset_index(drop=True)

    return df_basket

# Apply the function to each df_basket_# DataFrame
for cluster_id in cluster_dataframes.keys():
    cluster_dataframes[cluster_id] = filter_songs(cluster_dataframes[cluster_id])

    # Optionally, save back to dynamically created variables
    globals()[cluster_id] = cluster_dataframes[cluster_id]

## Compute association rules and support
- Support and association rules are computed for each basket data frame
- The minimum support used is 2%
- Association rules are computed using the Apriori algorithm to generate frequent single items, dublets, and triplets

In [104]:
# Function to calculate support for itemsets
def calculate_support(data, itemsets):
    itemset_counts = defaultdict(int)
    for basket in data['basket']:
        for itemset in itemsets:
            if set(itemset).issubset(set(basket)):
                itemset_counts[itemset] += 1
    total_baskets = len(data)
    support = {itemset: count / total_baskets for itemset, count in itemset_counts.items()}
    return support

# Apriori Algorithm for Size 2 and 3
def apriori_pairs_and_triplets(data, min_support):
    frequent_itemsets = {}

    # Start with single items
    items = set(item for basket in data['basket'] for item in basket)
    single_itemsets = [(item,) for item in items]
    
    # Calculate support for single items
    support = calculate_support(data, single_itemsets)
    frequent_singles = [itemset for itemset, sup in support.items() if sup >= min_support]
    frequent_itemsets.update({itemset: support[itemset] for itemset in frequent_singles})
    
    print(f"Frequent Single Items: {frequent_singles}")
    
    # Generate pairs (size 2)
    pairs = list(combinations(set(item for itemset in frequent_singles for item in itemset), 2))
    pair_support = calculate_support(data, pairs)
    frequent_pairs = [itemset for itemset, sup in pair_support.items() if sup >= min_support]
    frequent_itemsets.update({itemset: pair_support[itemset] for itemset in frequent_pairs})
    
    print(f"Frequent Pairs: {frequent_pairs}")
    
    # Generate triplets (size 3)
    triplets = list(combinations(set(item for itemset in frequent_pairs for item in itemset), 3))
    triplet_support = calculate_support(data, triplets)
    frequent_triplets = [itemset for itemset, sup in triplet_support.items() if sup >= min_support]
    frequent_itemsets.update({itemset: triplet_support[itemset] for itemset in frequent_triplets})
    
    print(f"Frequent Triplets: {frequent_triplets}")
    
    return frequent_itemsets

## Generate results (freq. items and support)
The frequent items and item sets with corresponding support are stored in a CSV file for each cluster.

In [105]:
# Create a dictionary for all baskets
df_baskets = {f"df_basket_{i}": globals()[f"df_basket_{i}"] for i in range(1, 12)}

min_support = 0.02

# Process each basket and save results
for i, (name, data) in enumerate(df_baskets.items(), start=1):
    print(f"Processing {name}...")
    frequent_itemsets = apriori_pairs_and_triplets(data, min_support)
    
    # Convert frequent itemsets to a DataFrame
    results_df = pd.DataFrame(
        [{"itemset": itemset, "support": support} for itemset, support in frequent_itemsets.items()]
    )
    
    # Save each result DataFrame with a unique name
    output_file = f"Data/Results/cluster_{i}_results.csv"  # Use the index 'i' for unique filenames
    results_df.to_csv(output_file, index=False)
    print(f"Results saved for cluster {i} to {output_file}")

Processing df_basket_1...
Frequent Single Items: [('TRBIZJO128F4259F57',), ('TRZFXSA128F93073BE',), ('TRUPDNU128F147CBE9',), ('TRPQPVO128F1460295',), ('TRBZHVK128F1460299',), ('TRBYSLP128F147CC94',), ('TRWSXPD128F4259F3B',), ('TRFMXJI128F930738B',), ('TRWKOEO128F147CC66',), ('TRHOEPA128F4259F3F',), ('TRUQERG128F147CBEA',), ('TRFLVKB128F147CC69',), ('TRBKSWQ128F4259F3C',), ('TRDKILY128F147CC86',), ('TRNRTQT128F4259F55',), ('TREAYDQ128F147CC6C',), ('TRJRIHX128F147CBC5',), ('TRRMNLW128F92D92B0',), ('TRUNYCG128F423A553',), ('TRUIIPM128F147CBD2',), ('TRYDIVP12903CAF5A7',), ('TRTAMNT128E07926F7',), ('TRAFMVN128F147CBCE',), ('TRDRTNQ128F93313A3',), ('TRVBFHW128F147CC96',), ('TRLNLYC128F422B8D2',), ('TRFCHEY128E079219C',), ('TRVLXMQ128F9339980',), ('TRXODKF128F9329E17',), ('TRPEGDQ128F423B914',), ('TRWIKPM128F931F1D2',), ('TRYQEML128F9320B6A',), ('TRUDXEG128F9345084',), ('TRSKQZQ12903CAF5A3',), ('TRILQPW128F92E2887',), ('TRPZMCK128F934C24E',), ('TREYLBG128F934B6EA',), ('TRZQTVH128F934B6E3',), 

## Visualize results: Final songs recommendations
- Heatmap showing the support between pairs of songs (duplets)
- Network graph showing triplets

In [107]:
# Placeholder similarity measure function using song attributes
def calculate_similarity(track_id1, track_id2, track_id_to_attributes):
    attributes1 = track_id_to_attributes.get(track_id1)
    attributes2 = track_id_to_attributes.get(track_id2)
    if attributes1 is not None and attributes2 is not None:
        # Compute cosine similarity between the attribute vectors
        similarity = cosine_similarity([attributes1], [attributes2])[0][0]
        return similarity
    else:
        return 0  # Or handle missing data appropriately

# Function to process a single file for heatmaps and graphs
def process_csv(file_path, id_to_name, track_id_to_attributes):
    # Load duplets and their support values
    df = pd.read_csv(file_path)
    
    # Parse the 'itemset' column safely
    duplets = {
        tuple(item.strip() for item in ast.literal_eval(row["itemset"])): row["support"]
        for _, row in df.iterrows()
    }

    # Filter out pairs including the track ID for 'Revelry' and ensure only pairs are included
    # Replace 'track_id_revelry' with the actual track ID for 'Revelry' if you wish to exclude it
    track_id_revelry = 'track_id_revelry'  # Placeholder; replace with actual track ID if needed

    filtered_duplets = {
        pair: support for pair, support in duplets.items()
        if track_id_revelry not in pair and len(pair) == 2
    }

    # Prepare data for support heatmap
    track_ids = sorted(list(set([item for pair in filtered_duplets.keys() for item in pair])))
    heatmap_data = pd.DataFrame(0, index=track_ids, columns=track_ids)

    for (track_id1, track_id2), support in filtered_duplets.items():
        heatmap_data.loc[track_id1, track_id2] = support
        heatmap_data.loc[track_id2, track_id1] = support  # Symmetric pairs

    # Map track IDs to song names for labeling
    track_id_to_name = {
        track_id: id_to_name.get(track_id, f"Unknown({track_id})")
        for track_id in track_ids
    }
    heatmap_data.rename(index=track_id_to_name, columns=track_id_to_name, inplace=True)

    # Save the support heatmap
    plt.figure(figsize=(12, 10))
    sns.heatmap(
        heatmap_data,
        annot=False,
        cmap="Blues",
        cbar_kws={'label': 'Support'}
    )
    plt.title(f"Song Pair Support Heatmap - {os.path.basename(file_path)}", fontsize=16)
    plt.xticks(rotation=45, ha='right', fontsize=10)
    plt.yticks(fontsize=10)
    plt.tight_layout()

    heatmap_path = os.path.join(output_dir_heatmaps, f"{os.path.basename(file_path).replace('.csv', '')}_heatmap.png")
    plt.savefig(heatmap_path)
    plt.close()
    print(f"Support heatmap saved to {heatmap_path}")

    # Prepare data for similarity heatmap using track IDs
    similarity_matrix = np.zeros((len(track_ids), len(track_ids)))
    for i, track_id1 in enumerate(track_ids):
        for j, track_id2 in enumerate(track_ids):
            if i <= j:  # Compute only upper triangular and diagonal
                similarity = calculate_similarity(track_id1, track_id2, track_id_to_attributes)
                similarity_matrix[i, j] = similarity
                similarity_matrix[j, i] = similarity  # Symmetric matrix

    # Create DataFrame for similarity heatmap
    similarity_df = pd.DataFrame(similarity_matrix, index=track_ids, columns=track_ids)
    # Map track IDs to song names for labeling
    similarity_df.rename(index=track_id_to_name, columns=track_id_to_name, inplace=True)

    # Save the similarity heatmap
    plt.figure(figsize=(12, 10))
    sns.heatmap(
        similarity_df,
        annot=False,
        cmap="coolwarm_r",
        cbar_kws={'label': 'Similarity'},
        xticklabels=True,
        yticklabels=True
    )
    plt.title(f"Song Similarity Heatmap - {os.path.basename(file_path)}", fontsize=16)
    plt.xticks(rotation=45, ha='right', fontsize=10)
    plt.yticks(fontsize=10)
    plt.tight_layout()

    similarity_heatmap_path = os.path.join(output_dir_heatmaps, f"{os.path.basename(file_path).replace('.csv', '')}_similarity_heatmap.png")
    plt.savefig(similarity_heatmap_path)
    plt.close()
    print(f"Similarity heatmap saved to {similarity_heatmap_path}")

In [108]:
# Directories for input and output
input_dir = "Data/Results"
output_dir_heatmaps = "Data/Heatmaps"
os.makedirs(output_dir_heatmaps, exist_ok=True)

# Define the mapping of track IDs to names
id_to_name = {
    track_id.strip(): name
    for track_id, name in df_songs.set_index('track_id')['name'].to_dict().items()
}

# Prepare a mapping from track IDs to their attributes (with placeholders)
# Let's assume the attributes are 'attr1', 'attr2', ..., 'attr6'
attribute_columns = ['danceability','loudness','speechiness','acousticness','instrumentalness','liveness','valence','tempo']

# Standardize the attributes
scaler = StandardScaler()
df_songs[attribute_columns] = scaler.fit_transform(df_songs[attribute_columns])

# Create a dictionary mapping track IDs to their attribute vectors
track_id_to_attributes = {
    track_id: df_songs.loc[df_songs['track_id'] == track_id, attribute_columns].values.flatten()
    for track_id in df_songs['track_id']
}

# Process each CSV file in the input directory
for file_name in os.listdir(input_dir):
    if file_name.endswith(".csv"):  # Process only CSV files
        file_path = os.path.join(input_dir, file_name)
        print(f"Processing {file_name}...")
        process_csv(file_path, id_to_name, track_id_to_attributes)

Processing cluster_4_results.csv...
Support heatmap saved to Data/Heatmaps/cluster_4_results_heatmap.png
Similarity heatmap saved to Data/Heatmaps/cluster_4_results_similarity_heatmap.png
Processing cluster_11_results.csv...
Support heatmap saved to Data/Heatmaps/cluster_11_results_heatmap.png
Similarity heatmap saved to Data/Heatmaps/cluster_11_results_similarity_heatmap.png
Processing cluster_8_results.csv...
Support heatmap saved to Data/Heatmaps/cluster_8_results_heatmap.png
Similarity heatmap saved to Data/Heatmaps/cluster_8_results_similarity_heatmap.png
Processing cluster_1_results.csv...
Support heatmap saved to Data/Heatmaps/cluster_1_results_heatmap.png
Similarity heatmap saved to Data/Heatmaps/cluster_1_results_similarity_heatmap.png
Processing cluster_2_results.csv...
Support heatmap saved to Data/Heatmaps/cluster_2_results_heatmap.png
Similarity heatmap saved to Data/Heatmaps/cluster_2_results_similarity_heatmap.png
Processing cluster_7_results.csv...
Support heatmap saved

In [98]:
def create_graph_from_results(file_path, id_to_name, output_dir_graphs):
    # Load the results file
    df = pd.read_csv(file_path)
    
    # Extract triplets and map track IDs to names
    triplets = {
        tuple(item.strip() for item in row["itemset"].strip("()").replace("'", "").split(", ")): row["support"]
        for _, row in df.iterrows()
    }
    triplets_with_names = [
        tuple(id_to_name.get(item, f"Unknown({item})") for item in triplet)
        for triplet in triplets.keys()
    ]

    # Create the graph
    G = nx.Graph()

    # Add edges for each triplet
    for triplet in triplets_with_names:
        for pair in combinations(triplet, 2):
            G.add_edge(pair[0], pair[1])

    # Identify connected components (clusters)
    groups = [G.subgraph(c).copy() for c in nx.connected_components(G)]

    # Plot each cluster
    for i, group in enumerate(groups):
        plt.figure(figsize=(10, 6))
        pos = nx.spring_layout(group, seed=42)  # Use spring layout for neat arrangement
        nx.draw_networkx_nodes(group, pos, node_size=1000, node_color='skyblue', alpha=0.9)
        nx.draw_networkx_edges(group, pos, width=2, edge_color='gray', alpha=0.6)
        nx.draw_networkx_labels(group, pos, font_size=10, font_color='black')
        plt.title(f"Group {i + 1} - {os.path.basename(file_path)}", fontsize=14)
        plt.axis('off')  # Turn off axes

        # Save the graph visualization
        graph_path = os.path.join(output_dir_graphs, f"{os.path.basename(file_path).replace('.csv', '')}_group_{i + 1}.png")
        plt.savefig(graph_path)
        plt.close()
        print(f"Graph saved for cluster {i} to {graph_path}")


In [99]:
# Example usage
output_dir_graphs = "Data/Graphs"
os.makedirs(output_dir_graphs, exist_ok=True)

id_to_name = {track_id.strip(): name for track_id, name in df_songs.set_index('track_id')['name'].to_dict().items()}  # Define mapping

for file_name in os.listdir(input_dir):
    if file_name.endswith(".csv"):  # Process only CSV files
        file_path = os.path.join(input_dir, file_name)
        print(f"Processing graph for {file_name}...")
        create_graph_from_results(file_path, id_to_name, output_dir_graphs)

Processing graph for cluster_1_results.csv...
Graph saved for cluster 0 to Data/Graphs/cluster_1_results_group_1.png
Graph saved for cluster 1 to Data/Graphs/cluster_1_results_group_2.png
Graph saved for cluster 2 to Data/Graphs/cluster_1_results_group_3.png
Graph saved for cluster 3 to Data/Graphs/cluster_1_results_group_4.png
Graph saved for cluster 4 to Data/Graphs/cluster_1_results_group_5.png
Graph saved for cluster 5 to Data/Graphs/cluster_1_results_group_6.png
Graph saved for cluster 6 to Data/Graphs/cluster_1_results_group_7.png
